<h1>Maximizing the spread of influence through a social network<h1>

In [1]:
import numpy as np
import random
from itertools import combinations
from collections import OrderedDict

<h3>1. Example to work on</h3>

In [2]:
graph = {'v1': ['v2', 'v3', 'v5'],
         'v2': [],
         'v3': ['v5'],
         'v4': ['v2', 'v3', 'v6'],
         'v5': ['v4'],
         'v6': ['v2', 'v5']}

<h3>2. Greedy algorithm application</h3>

In [3]:
#this function returns the set of arcs in the graph
def graph_arcs(graph):
  h = []
  for u in graph.keys():
    for v in graph[u]:
      h.append((u, v))
  return h

arcs = graph_arcs(graph) #set of arcs in the graph

In [4]:
#this function returns a list of all the possible sets of active arcs in our the graph according to a probability p_u
def active_arcs(arcs):
  H = [] 
  for n in range(len(arcs) + 1):  
    H += list(combinations(arcs, n))
  for i in range(len(H)):
    H[i] = list(H[i])
  return H #every list in H is a possible scenario of arcs activation

#H = active_arcs(arcs)[420]  #activation example in the presentation
#un autre exemple pour vérifier
#H = active_arcs(arcs)[529]
H_set = active_arcs(arcs)

In [5]:
#this function returns the possible paths from a given graph, starting from one vertex to reach another vertex
def find_all_paths(graph, start, end, path=[]):
        path = path + [start]
        if start == end:
            return [path]
        if start not in graph.keys():
            return []
        paths = []
        for node in graph[start]:
            if node not in path:
                newpaths = find_all_paths(graph, node, end, path)
                for newpath in newpaths:
                    paths.append(newpath)
        return paths

find_all_paths(graph, 'v1', 'v4')

[['v1', 'v3', 'v5', 'v4'], ['v1', 'v5', 'v4']]

In [6]:
#this function returns the active graph generated from active arcs
def active_graph(H): # H = list of active arcs
  V = []
  for arc in H:
    V.append(arc[0])
  V = list(set(V))
  D = dict.fromkeys(V)

  for v in D.keys():
    T = []
    for arcs in H:
      if v == arcs[0]:
        T.append(arcs[1])
    D[v] = T
  return D

#propagation_graph = active_graph(H)
#propagation_graph

In [7]:
#this function returns the influence of each vertex
def vertices_influence(propagation_graph):
  inf_vertex = dict()
  for v in graph.keys():
    list_of_activated_verteces_from_v = []
    
    for t in graph.keys():
      find_all_paths(propagation_graph, v, t)
      if len(find_all_paths(propagation_graph, v, t))!=0:
        list_of_activated_verteces_from_v.append(t)

    inf_vertex[v] = (list_of_activated_verteces_from_v, len(list_of_activated_verteces_from_v))
  return(inf_vertex)

#vertices_infl = vertices_influence(propagation_graph)
#vertices_infl

In [8]:
#this function add to S the most influencer vertex and return the set S.
def biggest_infl_to_S(vertices_infl, S):
  M = 0
  vm = 'O'
  for v in vertices_infl.keys():
    if vertices_infl[v][1] > M:
      M = vertices_infl[v][1]
      vm = v
  S.append((vm, M))
  return (S)

#selected_vertex = biggest_infl_to_S(vertices_infl, [])
#selected_vertex

In [9]:
#This function returns the number of vertices in the graph influenced by the vertices in S.
def S_infl(S, vertices_infl):
  S_influence = 0
  for el in S:
    S_influence = S_influence + vertices_infl[el][1]
  return S_influence

#This function returns the number of vertices in the graph influenced by the vertices in S union a vertex v influence.
def S_infl_U_v(S, v, vertices_infl):
  S_infl_U_v_influence = S_infl(S, vertices_infl)
  l = []
  for el in S:
    for iv in vertices_infl[el][0]:
      l.append(iv)
  for ele in vertices_infl[v][0]:
    if ele not in l:
      S_infl_U_v_influence = S_infl_U_v_influence + 1
  return S_infl_U_v_influence

#this function returns the list of seed vertices with its influence result
def greedy_alg(k):
  S = []
  S_with_infl_reslt = []
  for i in range(k):
    result = dict()
    vertices_infl = vertices_influence(propagation_graph)
        
    for v in vertices_infl.keys():
      if v not in S:
        result[v] = S_infl_U_v(S, v, vertices_infl) - S_infl(S, vertices_infl)
    S.append(max(result, key=result.get))
    S_with_infl_reslt.append((max(result, key=result.get), result[max(result, key=result.get)]))
    print("iteration ", i+1, " ==> Resulting seed vertices list : ", S_with_infl_reslt)
  print("The obtained seed vertices list : ", S_with_infl_reslt)
  return S_with_infl_reslt


#S = [t[0] for t in greedy_alg(2)]

<h3>3. Cascade Model application</h3>
In this part, using the initial graph, we select the arcs coming out of the vertices in S which are initialized as "unflipped", and according to a probability p we decide to activate or not each of the arcs.

In [10]:
#this function returns the graph containing only arcs and vertices composed using the returned list S by the greedy algorithm.
def graph_of_selected_vertices(graph, S):
  G = dict()
  for key, value in graph.items():
    if key in S:
      G[key] = value
  return G


#Cascade Model : In this function we execute the cascade model on the graph starting by activating the vertices in the list S.
def cascade_model(graph, S):
  active_vertices = S   #S is the latest active vertices
  graph_to_check = dict()
  itr = 0
  while graph_to_check != graph_of_selected_vertices(graph, active_vertices):
    print("__________ Iteration ",itr + 1 , " __________")
    print("--- Active vertices :", S, " ---")
    G = graph_of_selected_vertices(graph, S)
    for i in G.keys(): 
      graph_to_check[i] = G[i]

    #returning arcs (unflipped) in the new graph
    new_arcs = graph_arcs(G)
    new_H = active_arcs(new_arcs)

    S =[] #newly active vertices
    pile_ou_face = random.choice(new_H)
    print("Arcs activated using heads or tails : ", pile_ou_face)
    for t in pile_ou_face:
      active_vertices.append(t[1])
      S.append(t[1])
    active_vertices = list(set(active_vertices))
    print("newly active vertices in this iteration:", S)
    itr = itr + 1
  print("***********************************************************************")
  print("Active vertices after cascade model application : ", active_vertices)
  print("***********************************************************************")
  return active_vertices

<h3>4. Combination of Greedy algorithm and Cascade model</h3>
In this part we use the all the possible arcs activation applying the Greedy algorithm and the cascade model.

In [11]:
#this cell returns all the average seed vertices given by greedy algorithme applied on all the possibilities of arcs activation
k=2
S = []
all_gready_output = dict() #list of vertices every iteration

greedy_final_result = dict()  #final selected vertices
for v in graph.keys():
  greedy_final_result[v] = 0

for H in H_set:
  H_number = H_set.index(H)
  print("=====> activation arcs alternative number H *** ", H_number + 1, " *** using heads or tails : ")
  propagation_graph = active_graph(H)
  print("Satrting of Greedy algorithms execution...")
  S = [t[0] for t in greedy_alg(k)]
  for t in greedy_alg(k):
    greedy_final_result[t[0]] = greedy_final_result[t[0]] + t[1]
  all_gready_output[H_number] = [el for el in S]


greedy_final_result_descending = OrderedDict(sorted(greedy_final_result.items(), key=lambda kv: kv[1], reverse=True))
print("------------------------------------------------------------------------------------------")
print("Greedy algorithm final result descending : ", greedy_final_result_descending)

S = []
for el in range(k):
  S.append(list(greedy_final_result_descending.keys())[el])



Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
iteration  1  ==> Resulting seed vertices list :  [('v1', 4)]
iteration  2  ==> Resulting seed vertices list :  [('v1', 4), ('v4', 1)]
The obtained seed vertices list :  [('v1', 4), ('v4', 1)]
iteration  1  ==> Resulting seed vertices list :  [('v1', 4)]
iteration  2  ==> Resulting seed vertices list :  [('v1', 4), ('v4', 1)]
The obtained seed vertices list :  [('v1', 4), ('v4', 1)]
=====> activation arcs alternative number H ***  401  *** using heads or tails : 
Satrting of Greedy algorithms execution...
iteration  1  ==> Resulting seed vertices list :  [('v1', 4)]
iteration  2  ==> Resulting seed vertices list :  [('v1', 4), ('v4', 1)]
The obtained seed vertices list :  [('v1', 4), ('v4', 1)]
iteration  1  ==> Resulting seed vertices list :  [('v1', 4)]
iteration  2  ==> Resulting seed vertices list :  [('v1', 4), ('v4', 1)]
The obtained seed vertices list :  [('v1', 4), ('v4', 1)]
=====> activation arcs al

In [12]:
#Execution of cascade model on seed vertices
print("The seed vertices set containing ", k, "vertices : S = ", S)
print("===> Application of Cascade model on this gready algorithm output : ")
final_active_vertices = cascade_model(graph, S)

The seed vertices set containing  2 vertices : S =  ['v1', 'v4']
===> Application of Cascade model on this gready algorithm output : 
__________ Iteration  1  __________
--- Active vertices : ['v1', 'v4']  ---
Arcs activated using heads or tails :  [('v1', 'v2'), ('v1', 'v5'), ('v4', 'v3'), ('v4', 'v6')]
newly active vertices in this iteration: ['v2', 'v5', 'v3', 'v6']
__________ Iteration  2  __________
--- Active vertices : ['v2', 'v5', 'v3', 'v6']  ---
Arcs activated using heads or tails :  [('v3', 'v5'), ('v6', 'v2')]
newly active vertices in this iteration: ['v5', 'v2']
***********************************************************************
Active vertices after cascade model application :  ['v5', 'v2', 'v4', 'v3', 'v1', 'v6']
***********************************************************************


*Haithem BEN DRISSI*

*Département MIDO*

*Université Paris Dauphine-PSL*